In [1]:
!pip install apify-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 1.6 MB/s eta 0:00:00


In [2]:
import csv
from apify_client import ApifyClient
from datetime import datetime

In [ ]:
# Initialize the ApifyClient with your Apify API token
client = ApifyClient("apify_api_ahFfQ3XdhejhYby4v7AbYNLvzc6IdC3d8AL3")

In [ ]:
# List of TripAdvisor URLs for 100 Sri Lankan hotels
hotel_urls = [
    "https://www.tripadvisor.com/Hotel_Review-g297896-d14033359-Reviews-Le_Grand_Galle-Galle_Galle_District_Southern_Province.html",
    "https://www.tripadvisor.com/Hotel_Review-g293962-d14106301-Reviews-Marino_Beach_Colombo-Colombo_Western_Province.html"
    # Add up to 100 URLs for each hotel here
]

In [ ]:
# Prepare to collect all reviews in a list
all_reviews = []
target_review_count = 1000

In [ ]:
#Define the date range
start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 11, 2)

In [ ]:
for url in hotel_urls:
    # Prepare input for each hotel with English-only reviews
    run_input = {
        "startUrls": [{"url": url}],
        "maxItemsPerQuery": 50,  # Adjust if you need more reviews per query
        "reviewRatings": ["ALL_REVIEW_RATINGS"],
        "reviewsLanguages": ["en"],  # Only fetch English reviews
    }

    # Run the Actor and wait for it to finish
    run = client.actor("maxcopell/tripadvisor-reviews").call(run_input=run_input)

    # Fetch reviews from the dataset
    for item in client.dataset(run["defaultDatasetId"]).iterate_items():
        # Parse the review date
        review_date = datetime.strptime(item["publishedDate"], "%Y-%m-%d")

        # Filter reviews by date range
        if start_date <= review_date <= end_date:
            all_reviews.append(item)

        # Stop if we've reached the target review count
        if len(all_reviews) >= target_review_count:
            break

    # Check if we've collected enough reviews
    if len(all_reviews) >= target_review_count:
        break


In [ ]:
# Define the CSV file path and field names
csv_file_path = "sri_lankan_hotel_reviews.csv"
fieldnames = ["reviewId", "userName", "publishedDate", "rating", "reviewText"]

In [ ]:
# Write the filtered reviews to the CSV file
with open(csv_file_path, mode="w", newline="", encoding="utf-8") as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()  # Write header

    for review in all_reviews:
        # Extract only available fields to avoid missing data issues
        writer.writerow({
            "reviewId": review.get("reviewId", ""),
            "userName": review.get("userName", ""),
            "publishedDate": review.get("publishedDate", ""),
            "rating": review.get("rating", ""),
            "reviewText": review.get("reviewText", ""),
        })

print(f"Collected {len(all_reviews)} English reviews in the specified date range.")
print(f"Data saved to {csv_file_path}")